In [9]:
import pandas as pd
import geopandas as gpd
import numpy as np
from geoalchemy2 import Geometry, WKTElement
from sqlalchemy import *
import psycopg2
import time
%matplotlib inline

pd.options.display.max_rows = 999
pd.options.display.max_columns = None

In [10]:
password = open('password.txt','r').readline()
username = 'kmerrem'

#this is to write from geopandas to postgis table 
#gisdata_engine = create_engine('postgresql://%s:%s@172.19.118.205:5432/gisdata' %(username,password))
walkability_engine = create_engine('postgresql://%s:%s@172.19.118.205:5432/walkability' %(username,password))

In [11]:


buildings = gpd.read_postgis("select * from final.buildings", walkability_engine, 'geom')
cts = gpd.read_postgis("SELECT * FROM public.census_tracts_blank", walkability_engine, 'geom')
#pednet = gpd.read_postgis("select * from compliance.att_pednet_all_in WHERE geom IS NOT NULL", walkability_engine, 'geom')
#pednet = gpd.read_postgis("select * from staging.a_z1_pednet WHERE geom IS NOT NULL", walkability_engine, 'geom')
pednet = gpd.read_postgis("select * from final.pednet WHERE geom IS NOT NULL", walkability_engine, 'geom')

In [12]:
buildings.head(2)

,geom,id,ctuid,ctname,ctnum,geotext,d_fc_0,n_fc_0,d_ff_0,n_ff_0,d_sm_0,n_sm_0,d_hp_0,n_hp_0,d_sxhc_0,n_sxhc_0,d_wc_0,n_wc_0,d_sc_0,n_sc_0,d_ar_0,n_ar_0,d_dc_0,n_dc_0,d_di_0,n_di_0,d_ems_0,n_ems_0,d_fc_1,n_fc_1,d_ff_1,n_ff_1,d_sm_1,n_sm_1,d_hp_1,n_hp_1,d_sxhc_1,n_sxhc_1,d_wc_1,n_wc_1,d_sc_1,n_sc_1,d_ar_1,n_ar_1,d_dc_1,n_dc_1,d_di_1,n_di_1,d_ems_1,n_ems_1,d_frc_0,n_frc_0,d_fr_0,n_fr_0,d_lb_0,n_lb_0,d_plc_0,n_plc_0,d_rec_0,n_rec_0,d_thing_0,n_thing_0,d_vtl_0,n_vtl_0,d_bp_0,n_bp_0,d_bs_0,n_bs_0,d_ttcstop_0,n_ttcstop_0,d_ttcst_0,n_ttcst_0,d_ttcacc_0,n_ttcacc_0,d_frc_1,n_frc_1,d_fr_1,n_fr_1,d_lb_1,n_lb_1,d_plc_1,n_plc_1,d_rec_1,n_rec_1,d_thing_1,n_thing_1,d_vtl_1,n_vtl_1,d_bp_1,n_bp_1,d_bs_1,n_bs_1,d_ttcstop_1,n_ttcstop_1,d_ttcst_1,n_ttcst_1,d_ttcacc_1,n_ttcacc_1,m_fc_0,m_fc_1,m_ff_0,m_ff_1,m_sm_0,m_sm_1,m_hp_0,m_hp_1,m_sxhc_0,m_sxhc_1,m_wc_0,m_wc_1,m_sc_0,m_sc_1,m_ar_0,m_ar_1,m_dc_0,m_dc_1,m_di_0,m_di_1,m_ems_0,m_ems_1,m_frc_0,m_frc_1,m_fr_0,m_fr_1,m_lb_0,m_lb_1,m_plc_0,m_plc_1,m_rec_0,m_rec_1,m_thing_0,m_thing_1,m_vtl_0,m_vtl_1,m_bp_0,m_bp_1,m_bs_0,m_bs_1,m_ttcstop_0,m_ttcstop_1,m_ttcst_0,m_ttcst_1,m_ttcacc_0,m_ttcacc_1,n_fc_0n,n_fc_0type,n_ff_0n,n_ff_0type,n_sm_0n,n_sm_0type,n_hp_0n,n_hp_0type,n_sxhc_0n,n_sxhc_0type,n_wc_0n,n_wc_0type,n_sc_0n,n_ar_0n,n_ar_0type,n_lb_0n,n_dc_0n,n_dc_0type,n_fr_0n,n_fr_0type,n_di_0n,n_dc_0type_1,n_ems_0n,n_ems_0type,n_frc_0n,n_frc_0type,n_plc_0n,n_plc_0type,n_rec_0n,n_rec_0type,n_thing_0n,n_thing_0type,n_vtl_0n,n_vtl_0type,n_bp_0type,n_bs_0type,n_ttcstop_0n,n_ttcstop_0type,n_ttcst_0n,n_ttcst_0type,n_ttcacc_0n,n_ttcacc_0type,n_fc_1n,n_fc_1type,n_ff_1n,n_ff_1type,n_sm_1n,n_sm_1type,n_hp_1n,n_hp_1type,n_sxhc_1n,n_lb_1type,n_wc_1n,n_wc_1type,n_sc_1n,n_ar_1n,n_dc_1n,n_dc_1type,n_lb_1n,n_fr_1n,n_fr_1type,n_di_1n,n_dc_1type_1,n_ems_1n,n_ems_1type,n_frc_1n,n_frc_1type,n_plc_1n,n_plc_1type,n_rec_1n,n_rec_1type,n_thing_1n,n_thing_1type,n_vtl_1n,n_vtl_1type,n_bp_1type,n_bs_1type,n_ttcstop_1n,n_ttcstop_1type,n_ttcst_1n,n_ttcst_1type,n_ttcacc_1n,n_ttcacc_1type
0,"(POLYGON ((323669.8799999999 4844969.168, 3236...",73902,5350353.02,353.02,535035302.0,535035302,745.987000,2158.0,2205.412109,2696.0,930.328003,320.0,2216.054932,17.0,3968.365967,10.0,2815.935059,7.0,398.583008,54.0,1880.193970,28.0,580.385010,616.0,2633.145020,38.0,1307.880005,19.0,799.567017,134.0,2205.412109,3359.0,994.203003,449.0,5000.0,NaN,5000.0,NaN,2896.327881,50.0,580.385010,200.0,2171.842041,47.0,597.604004,179.0,2714.940918,37.0,1669.354004,13.0,1608.171997,107.0,1172.843994,10.0,2117.315918,43.0,2914.177002,19.0,1236.166016,100.0,1981.036987,45.0,398.583008,1471.0,3859.504883,87.0,5000.0,NaN,221.123993,5959.0,1208.963989,7.0,2174.330078,5.0,1608.171997,173.0,1860.125977,2.0,2361.991943,76.0,2965.291016,28.0,1314.344971,34.0,3859.504883,93.0,740.702026,1287.0,5000.0,NaN,5000.0,None,293.917999,3444.0,2174.330078,9.0,4111.562988,3.0,10.360931,11.105097,30.630724,30.630724,12.921222,13.808375,30.778541,69.444444,55.116194,69.444444,39.110209,40.226776,5.535875,8.060903,26.113805,30.164473,8.060903,8.300056,36.571459,37.707513,18.165000,23.185472,22.335722,22.335722,16.289500,25.835083,29.407166,32.805444,40.474681,41.184597,17.168972,18.254791,27.514403,53.604234,5.535875,10.287528,53.604234,69.444444,69.444444,69.444444,3.071167,4.082194,16.791167,30.199029,30.199029,57.105042,Midland Convenience,cs,None,None,TOP FOOD SUPERMARKET,sm,Rouge Valley Health System - Rouge Valley Cent...,h,Rexdale Youth Resource Center,Etobicoke North (1),Appletree Medical Centre - North York (York La...,w,CHINE DRIVE PUBLIC SCHOOL,Memorial Park,Toronto,McGregor Park Public Library,Plasp - Steelesview Public School,Steelesview Public School,FIRE STATION 231,740 MARKHAM RD,Scarborough Drop-In,The Church of the Epiphany,EMS Stn 43,Ambulance Station 43,Les Groupes Parent-Bambins,Centre francophone de Toronto,33 Division,50 Upjohn Road,Edithvale Park C.C.,Recreation Centres,Toronto Aerospace Museum,Museums,None,Eglinton Ave E,Bike Rack,None,WARDEN AVE AT APPLE CREEK BL

In [13]:
#jspatial joining of pednet to get ctnum on pednet segments
pednet_ct = gpd.sjoin(pednet,cts)


In [14]:
pednet_ct.head(2)

,gid_left,fid_1,__gid_left,linear_n_2,obs_pts,road_type,sdwlk_desc,sdwlk_code,crosswalk,zid,road_type1,flag1,stone,flag2,leng,f_cwalk,f_rd_type,px,px_type,cwalk_type,rd_typ,fstone,status,geom,crosswalk_boolean,index_right,gid_right,__gid_right,ctuid,ctname,pruid,prname,cmauid,cmapuid,cmaname,cmatype,shape_leng,shape_area,ctnum,geotext
0,107403,10731.0,82774.0,Unwin Avenue,None,Local,No sidewalk on either side,3.0,None,10407.0,None,LO,LO,None,53834.0,None,Local,None,None,None,Local,LO,DONE,"(LINESTRING (318125.824 4833673.8743, 318551.8...",None,0,1,62,5350001.00,0001.00,35,Ontario,535,35535,Toronto,B,36906.495474,6.619690e+06,535000100.0,535000100
80,117847,76087.0,63592.0,Ln 2 S Eastern W Logan,None,Laneway,Laneway without any sidewalks,10.0,None,76133.0,None,NDY,None,None,4880.0,None,Laneway,None,None,None,Laneway,LA,DONE,"(LINESTRING (317559.4142 4834849.1415, 317606....",None,0,1,62,5350001.00,0001.00,35,Ontario,535,35535,Toronto,B,36906.495474,6.619690e+06,535000100.0,535000100


In [15]:
#need to keep left join gid
pednet_sum = pednet_ct[['gid_left','ctuid','ctnum','geom',]]
pednet_sum.head(2)

,gid_left,ctuid,ctnum,geom
0,107403,5350001.00,535000100.0,"(LINESTRING (318125.824 4833673.8743, 318551.8..."
80,117847,5350001.00,535000100.0,"(LINESTRING (317559.4142 4834849.1415, 317606...."


In [16]:
summary_buildings  = buildings[['id','ctuid','ctnum','geom']]
summary_buildings.head(2)

,id,ctuid,ctnum,geom
0,73902,5350353.02,535035302.0,"(POLYGON ((323669.8799999999 4844969.168, 3236..."
1,73903,5350353.02,535035302.0,(POLYGON ((324253.4107999997 4844973.746400001...


In [17]:
#NEED THE INDEX TO RUN
cts =cts.set_index('ctnum')
#summary_buildings = summary_buildings.set_index('ctnum')

In [18]:
cts.head(5)

,gid,__gid,ctuid,ctname,pruid,prname,cmauid,cmapuid,cmaname,cmatype,shape_leng,shape_area,geotext,geom
ctnum,,,,,,,,,,,,,,
535000100.0,1,62,5350001.00,0001.00,35,Ontario,535,35535,Toronto,B,36906.495474,6.619690e+06,535000100,(POLYGON ((320588.9306980086 4835855.647789237...
535000200.0,2,63,5350002.00,0002.00,35,Ontario,535,35535,Toronto,B,32534.307654,3.293422e+06,535000200,"(POLYGON ((316642.831678931 4832470.554474635,..."
535026400.0,3,87,5350264.00,0264.00,35,Ontario,535,35535,Toronto,B,12241.116870,3.464007e+06,535026400,(POLYGON ((314351.8668838083 4845056.100542194...
535008200.0,4,94,5350082.00,0082.00,35,Ontario,535,35535,Toronto,B,2310.602463,3.082930e+05,535008200,(POLYGON ((318379.9191213865 4837764.438619476...
535000600.0,5,105,5350006.00,0006.00,35,Ontario,535,35535,Toronto,B,455.941409,1.287364e+04,535000600,"(POLYGON ((310234.199135254 4832619.302139398,..."


In [19]:
summary_buildings.head(2)

,id,ctuid,ctnum,geom
0,73902,5350353.02,535035302.0,"(POLYGON ((323669.8799999999 4844969.168, 3236..."
1,73903,5350353.02,535035302.0,(POLYGON ((324253.4107999997 4844973.746400001...


In [20]:
# writes blank minute columns _avg,_med,_min,_max to cts, summary_buildings and pednet_sum
out_columns = []
for measure in ['avg','med','min','max']:
    for current_column in buildings.columns:
        if current_column.startswith("m"):
            print(current_column)
            metric,poi,n = current_column.split("_")
            out_column = metric+"_"+poi+"_"+str(n)+"_"+measure
            out_columns.append(out_column)
            cts[out_column] = np.nan
            summary_buildings[out_column] = np.nan
            pednet_sum[out_column] = np.nan

m_fc_0
m_fc_1


/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


m_ff_0
m_ff_1
m_sm_0
m_sm_1
m_hp_0
m_hp_1
m_sxhc_0
m_sxhc_1
m_wc_0
m_wc_1
m_sc_0
m_sc_1
m_ar_0
m_ar_1
m_dc_0
m_dc_1
m_di_0
m_di_1
m_ems_0
m_ems_1
m_frc_0
m_frc_1
m_fr_0
m_fr_1
m_lb_0
m_lb_1
m_plc_0
m_plc_1
m_rec_0
m_rec_1
m_thing_0
m_thing_1
m_vtl_0
m_vtl_1
m_bp_0
m_bp_1
m_bs_0
m_bs_1
m_ttcstop_0
m_ttcstop_1
m_ttcst_0
m_ttcst_1
m_ttcacc_0
m_ttcacc_1
m_fc_0
m_fc_1
m_ff_0
m_ff_1
m_sm_0
m_sm_1
m_hp_0
m_hp_1
m_sxhc_0
m_sxhc_1
m_wc_0
m_wc_1
m_sc_0
m_sc_1
m_ar_0
m_ar_1
m_dc_0
m_dc_1
m_di_0
m_di_1
m_ems_0
m_ems_1
m_frc_0
m_frc_1
m_fr_0
m_fr_1
m_lb_0
m_lb_1
m_plc_0
m_plc_1
m_rec_0
m_rec_1
m_thing_0
m_thing_1
m_vtl_0
m_vtl_1
m_bp_0
m_bp_1
m_bs_0
m_bs_1
m_ttcstop_0
m_ttcstop_1
m_ttcst_0
m_ttcst_1
m_ttcacc_0
m_ttcacc_1
m_fc_0
m_fc_1
m_ff_0
m_ff_1
m_sm_0
m_sm_1
m_hp_0
m_hp_1
m_sxhc_0
m_sxhc_1
m_wc_0
m_wc_1
m_sc_0
m_sc_1
m_ar_0
m_ar_1
m_dc_0
m_dc_1
m_di_0
m_di_1
m_ems_0
m_ems_1
m_frc_0
m_frc_1
m_fr_0
m_fr_1
m_lb_0
m_lb_1
m_plc_0
m_plc_1
m_rec_0
m_rec_1
m_thing_0
m_thing_1
m_vtl_0
m_vtl_1
m_bp_0
m_b

In [21]:
buildings.head(2)

,geom,id,ctuid,ctname,ctnum,geotext,d_fc_0,n_fc_0,d_ff_0,n_ff_0,d_sm_0,n_sm_0,d_hp_0,n_hp_0,d_sxhc_0,n_sxhc_0,d_wc_0,n_wc_0,d_sc_0,n_sc_0,d_ar_0,n_ar_0,d_dc_0,n_dc_0,d_di_0,n_di_0,d_ems_0,n_ems_0,d_fc_1,n_fc_1,d_ff_1,n_ff_1,d_sm_1,n_sm_1,d_hp_1,n_hp_1,d_sxhc_1,n_sxhc_1,d_wc_1,n_wc_1,d_sc_1,n_sc_1,d_ar_1,n_ar_1,d_dc_1,n_dc_1,d_di_1,n_di_1,d_ems_1,n_ems_1,d_frc_0,n_frc_0,d_fr_0,n_fr_0,d_lb_0,n_lb_0,d_plc_0,n_plc_0,d_rec_0,n_rec_0,d_thing_0,n_thing_0,d_vtl_0,n_vtl_0,d_bp_0,n_bp_0,d_bs_0,n_bs_0,d_ttcstop_0,n_ttcstop_0,d_ttcst_0,n_ttcst_0,d_ttcacc_0,n_ttcacc_0,d_frc_1,n_frc_1,d_fr_1,n_fr_1,d_lb_1,n_lb_1,d_plc_1,n_plc_1,d_rec_1,n_rec_1,d_thing_1,n_thing_1,d_vtl_1,n_vtl_1,d_bp_1,n_bp_1,d_bs_1,n_bs_1,d_ttcstop_1,n_ttcstop_1,d_ttcst_1,n_ttcst_1,d_ttcacc_1,n_ttcacc_1,m_fc_0,m_fc_1,m_ff_0,m_ff_1,m_sm_0,m_sm_1,m_hp_0,m_hp_1,m_sxhc_0,m_sxhc_1,m_wc_0,m_wc_1,m_sc_0,m_sc_1,m_ar_0,m_ar_1,m_dc_0,m_dc_1,m_di_0,m_di_1,m_ems_0,m_ems_1,m_frc_0,m_frc_1,m_fr_0,m_fr_1,m_lb_0,m_lb_1,m_plc_0,m_plc_1,m_rec_0,m_rec_1,m_thing_0,m_thing_1,m_vtl_0,m_vtl_1,m_bp_0,m_bp_1,m_bs_0,m_bs_1,m_ttcstop_0,m_ttcstop_1,m_ttcst_0,m_ttcst_1,m_ttcacc_0,m_ttcacc_1,n_fc_0n,n_fc_0type,n_ff_0n,n_ff_0type,n_sm_0n,n_sm_0type,n_hp_0n,n_hp_0type,n_sxhc_0n,n_sxhc_0type,n_wc_0n,n_wc_0type,n_sc_0n,n_ar_0n,n_ar_0type,n_lb_0n,n_dc_0n,n_dc_0type,n_fr_0n,n_fr_0type,n_di_0n,n_dc_0type_1,n_ems_0n,n_ems_0type,n_frc_0n,n_frc_0type,n_plc_0n,n_plc_0type,n_rec_0n,n_rec_0type,n_thing_0n,n_thing_0type,n_vtl_0n,n_vtl_0type,n_bp_0type,n_bs_0type,n_ttcstop_0n,n_ttcstop_0type,n_ttcst_0n,n_ttcst_0type,n_ttcacc_0n,n_ttcacc_0type,n_fc_1n,n_fc_1type,n_ff_1n,n_ff_1type,n_sm_1n,n_sm_1type,n_hp_1n,n_hp_1type,n_sxhc_1n,n_lb_1type,n_wc_1n,n_wc_1type,n_sc_1n,n_ar_1n,n_dc_1n,n_dc_1type,n_lb_1n,n_fr_1n,n_fr_1type,n_di_1n,n_dc_1type_1,n_ems_1n,n_ems_1type,n_frc_1n,n_frc_1type,n_plc_1n,n_plc_1type,n_rec_1n,n_rec_1type,n_thing_1n,n_thing_1type,n_vtl_1n,n_vtl_1type,n_bp_1type,n_bs_1type,n_ttcstop_1n,n_ttcstop_1type,n_ttcst_1n,n_ttcst_1type,n_ttcacc_1n,n_ttcacc_1type
0,"(POLYGON ((323669.8799999999 4844969.168, 3236...",73902,5350353.02,353.02,535035302.0,535035302,745.987000,2158.0,2205.412109,2696.0,930.328003,320.0,2216.054932,17.0,3968.365967,10.0,2815.935059,7.0,398.583008,54.0,1880.193970,28.0,580.385010,616.0,2633.145020,38.0,1307.880005,19.0,799.567017,134.0,2205.412109,3359.0,994.203003,449.0,5000.0,NaN,5000.0,NaN,2896.327881,50.0,580.385010,200.0,2171.842041,47.0,597.604004,179.0,2714.940918,37.0,1669.354004,13.0,1608.171997,107.0,1172.843994,10.0,2117.315918,43.0,2914.177002,19.0,1236.166016,100.0,1981.036987,45.0,398.583008,1471.0,3859.504883,87.0,5000.0,NaN,221.123993,5959.0,1208.963989,7.0,2174.330078,5.0,1608.171997,173.0,1860.125977,2.0,2361.991943,76.0,2965.291016,28.0,1314.344971,34.0,3859.504883,93.0,740.702026,1287.0,5000.0,NaN,5000.0,None,293.917999,3444.0,2174.330078,9.0,4111.562988,3.0,10.360931,11.105097,30.630724,30.630724,12.921222,13.808375,30.778541,69.444444,55.116194,69.444444,39.110209,40.226776,5.535875,8.060903,26.113805,30.164473,8.060903,8.300056,36.571459,37.707513,18.165000,23.185472,22.335722,22.335722,16.289500,25.835083,29.407166,32.805444,40.474681,41.184597,17.168972,18.254791,27.514403,53.604234,5.535875,10.287528,53.604234,69.444444,69.444444,69.444444,3.071167,4.082194,16.791167,30.199029,30.199029,57.105042,Midland Convenience,cs,None,None,TOP FOOD SUPERMARKET,sm,Rouge Valley Health System - Rouge Valley Cent...,h,Rexdale Youth Resource Center,Etobicoke North (1),Appletree Medical Centre - North York (York La...,w,CHINE DRIVE PUBLIC SCHOOL,Memorial Park,Toronto,McGregor Park Public Library,Plasp - Steelesview Public School,Steelesview Public School,FIRE STATION 231,740 MARKHAM RD,Scarborough Drop-In,The Church of the Epiphany,EMS Stn 43,Ambulance Station 43,Les Groupes Parent-Bambins,Centre francophone de Toronto,33 Division,50 Upjohn Road,Edithvale Park C.C.,Recreation Centres,Toronto Aerospace Museum,Museums,None,Eglinton Ave E,Bike Rack,None,WARDEN AVE AT APPLE CREEK BL

In [22]:
cts.head(2)

,gid,__gid,ctuid,ctname,pruid,prname,cmauid,cmapuid,cmaname,cmatype,shape_leng,shape_area,geotext,geom,m_fc_0_avg,m_fc_1_avg,m_ff_0_avg,m_ff_1_avg,m_sm_0_avg,m_sm_1_avg,m_hp_0_avg,m_hp_1_avg,m_sxhc_0_avg,m_sxhc_1_avg,m_wc_0_avg,m_wc_1_avg,m_sc_0_avg,m_sc_1_avg,m_ar_0_avg,m_ar_1_avg,m_dc_0_avg,m_dc_1_avg,m_di_0_avg,m_di_1_avg,m_ems_0_avg,m_ems_1_avg,m_frc_0_avg,m_frc_1_avg,m_fr_0_avg,m_fr_1_avg,m_lb_0_avg,m_lb_1_avg,m_plc_0_avg,m_plc_1_avg,m_rec_0_avg,m_rec_1_avg,m_thing_0_avg,m_thing_1_avg,m_vtl_0_avg,m_vtl_1_avg,m_bp_0_avg,m_bp_1_avg,m_bs_0_avg,m_bs_1_avg,m_ttcstop_0_avg,m_ttcstop_1_avg,m_ttcst_0_avg,m_ttcst_1_avg,m_ttcacc_0_avg,m_ttcacc_1_avg,m_fc_0_med,m_fc_1_med,m_ff_0_med,m_ff_1_med,m_sm_0_med,m_sm_1_med,m_hp_0_med,m_hp_1_med,m_sxhc_0_med,m_sxhc_1_med,m_wc_0_med,m_wc_1_med,m_sc_0_med,m_sc_1_med,m_ar_0_med,m_ar_1_med,m_dc_0_med,m_dc_1_med,m_di_0_med,m_di_1_med,m_ems_0_med,m_ems_1_med,m_frc_0_med,m_frc_1_med,m_fr_0_med,m_fr_1_med,m_lb_0_med,m_lb_1_med,m_plc_0_med,m_plc_1_med,m_rec_0_med,m_rec_1_med,m_thing_0_med,m_thing_1_med,m_vtl_0_med,m_vtl_1_med,m_bp_0_med,m_bp_1_med,m_bs_0_med,m_bs_1_med,m_ttcstop_0_med,m_ttcstop_1_med,m_ttcst_0_med,m_ttcst_1_med,m_ttcacc_0_med,m_ttcacc_1_med,m_fc_0_min,m_fc_1_min,m_ff_0_min,m_ff_1_min,m_sm_0_min,m_sm_1_min,m_hp_0_min,m_hp_1_min,m_sxhc_0_min,m_sxhc_1_min,m_wc_0_min,m_wc_1_min,m_sc_0_min,m_sc_1_min,m_ar_0_min,m_ar_1_min,m_dc_0_min,m_dc_1_min,m_di_0_min,m_di_1_min,m_ems_0_min,m_ems_1_min,m_frc_0_min,m_frc_1_min,m_fr_0_min,m_fr_1_min,m_lb_0_min,m_lb_1_min,m_plc_0_min,m_plc_1_min,m_rec_0_min,m_rec_1_min,m_thing_0_min,m_thing_1_min,m_vtl_0_min,m_vtl_1_min,m_bp_0_min,m_bp_1_min,m_bs_0_min,m_bs_1_min,m_ttcstop_0_min,m_ttcstop_1_min,m_ttcst_0_min,m_ttcst_1_min,m_ttcacc_0_min,m_ttcacc_1_min,m_fc_0_max,m_fc_1_max,m_ff_0_max,m_ff_1_max,m_sm_0_max,m_sm_1_max,m_hp_0_max,m_hp_1_max,m_sxhc_0_max,m_sxhc_1_max,m_wc_0_max,m_wc_1_max,m_sc_0_max,m_sc_1_max,m_ar_0_max,m_ar_1_max,m_dc_0_max,m_dc_1_max,m_di_0_max,m_di_1_max,m_ems_0_max,m_ems_1_max,m_frc_0_max,m_frc_1_max,m_fr_0_max,m_fr_1_max,m_lb_0_max,m_lb_1_max,m_plc_0_max,m_plc_1_max,m_rec_0_max,m_rec_1_max,m_thing_0_max,m_thing_1_max,m_vtl_0_max,m_vtl_1_max,m_bp_0_max,m_bp_1_max,m_bs_0_max,m_bs_1_max,m_ttcstop_0_max,m_ttcstop_1_max,m_ttcst_0_max,m_ttcst_1_max,m_ttcacc_0_max,m_ttcacc_1_max
ctnum,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
535000100.0,1,62,5350001.00,0001.00,35,Ontario,535,35535,Toronto,B,36906.495474,6.619690e+06,535000100,(POLYGON ((320588.9306980086 4835855.647789237...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
535000200.0,2,63,5350002.00,0002.00,35,Ontario,535,35535,Toronto,B,32534.307654,3.293422e+06,535000200,"(POLYGON ((316642.831678931 4832470.554474635,...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [45]:
summary_buildings.head(2)

,id,ctuid,ctnum,geom,m_fc_0_avg,m_fc_1_avg,m_ff_0_avg,m_ff_1_avg,m_sm_0_avg,m_sm_1_avg,m_hp_0_avg,m_hp_1_avg,m_sxhc_0_avg,m_sxhc_1_avg,m_wc_0_avg,m_wc_1_avg,m_sc_0_avg,m_sc_1_avg,m_ar_0_avg,m_ar_1_avg,m_dc_0_avg,m_dc_1_avg,m_di_0_avg,m_di_1_avg,m_ems_0_avg,m_ems_1_avg,m_frc_0_avg,m_frc_1_avg,m_fr_0_avg,m_fr_1_avg,m_lb_0_avg,m_lb_1_avg,m_plc_0_avg,m_plc_1_avg,m_rec_0_avg,m_rec_1_avg,m_thing_0_avg,m_thing_1_avg,m_vtl_0_avg,m_vtl_1_avg,m_bp_0_avg,m_bp_1_avg,m_bs_0_avg,m_bs_1_avg,m_ttcstop_0_avg,m_ttcstop_1_avg,m_ttcst_0_avg,m_ttcst_1_avg,m_ttcacc_0_avg,m_ttcacc_1_avg,m_fc_0_med,m_fc_1_med,m_ff_0_med,m_ff_1_med,m_sm_0_med,m_sm_1_med,m_hp_0_med,m_hp_1_med,m_sxhc_0_med,m_sxhc_1_med,m_wc_0_med,m_wc_1_med,m_sc_0_med,m_sc_1_med,m_ar_0_med,m_ar_1_med,m_dc_0_med,m_dc_1_med,m_di_0_med,m_di_1_med,m_ems_0_med,m_ems_1_med,m_frc_0_med,m_frc_1_med,m_fr_0_med,m_fr_1_med,m_lb_0_med,m_lb_1_med,m_plc_0_med,m_plc_1_med,m_rec_0_med,m_rec_1_med,m_thing_0_med,m_thing_1_med,m_vtl_0_med,m_vtl_1_med,m_bp_0_med,m_bp_1_med,m_bs_0_med,m_bs_1_med,m_ttcstop_0_med,m_ttcstop_1_med,m_ttcst_0_med,m_ttcst_1_med,m_ttcacc_0_med,m_ttcacc_1_med,m_fc_0_min,m_fc_1_min,m_ff_0_min,m_ff_1_min,m_sm_0_min,m_sm_1_min,m_hp_0_min,m_hp_1_min,m_sxhc_0_min,m_sxhc_1_min,m_wc_0_min,m_wc_1_min,m_sc_0_min,m_sc_1_min,m_ar_0_min,m_ar_1_min,m_dc_0_min,m_dc_1_min,m_di_0_min,m_di_1_min,m_ems_0_min,m_ems_1_min,m_frc_0_min,m_frc_1_min,m_fr_0_min,m_fr_1_min,m_lb_0_min,m_lb_1_min,m_plc_0_min,m_plc_1_min,m_rec_0_min,m_rec_1_min,m_thing_0_min,m_thing_1_min,m_vtl_0_min,m_vtl_1_min,m_bp_0_min,m_bp_1_min,m_bs_0_min,m_bs_1_min,m_ttcstop_0_min,m_ttcstop_1_min,m_ttcst_0_min,m_ttcst_1_min,m_ttcacc_0_min,m_ttcacc_1_min,m_fc_0_max,m_fc_1_max,m_ff_0_max,m_ff_1_max,m_sm_0_max,m_sm_1_max,m_hp_0_max,m_hp_1_max,m_sxhc_0_max,m_sxhc_1_max,m_wc_0_max,m_wc_1_max,m_sc_0_max,m_sc_1_max,m_ar_0_max,m_ar_1_max,m_dc_0_max,m_dc_1_max,m_di_0_max,m_di_1_max,m_ems_0_max,m_ems_1_max,m_frc_0_max,m_frc_1_max,m_fr_0_max,m_fr_1_max,m_lb_0_max,m_lb_1_max,m_plc_0_max,m_plc_1_max,m_rec_0_max,m_rec_1_max,m_thing_0_max,m_thing_1_max,m_vtl_0_max,m_vtl_1_max,m_bp_0_max,m_bp_1_max,m_bs_0_max,m_bs_1_max,m_ttcstop_0_max,m_ttcstop_1_max,m_ttcst_0_max,m_ttcst_1_max,m_ttcacc_0_max,m_ttcacc_1_max
0,73902,5350353.02,535035302.0,"(POLYGON ((323669.8799999999 4844969.168, 3236...",9.86,12.53,36.42,36.42,14.3,15.88,25.11,69.31,50.02,69.44,37.99,44.58,6.3,9.01,27.83,36.13,7.19,9.8,40.1,41.23,16.82,26.06,13.99,19.42,21.06,27.42,23.23,34.64,42.92,47.85,12.65,21.67,22.41,48.5,6.42,9.27,48.51,69.44,69.44,69.44,3.97,4.89,23.29,32.22,32.12,52.07,10.1,12.85,36.73,36.73,14.65,16.27,25.12,69.44,50.34,69.44,38.51,45.05,6.28,9.01,28.39,36.84,7.3,9.67,41.22,42.36,17.81,26.64,12.48,19.72,21.04,27.97,22.26,34.28,44.14,48.42,12.61,21.62,22.7,48.83,6.49,9.41,48.83,69.44,69.44,69.44,3.82,5.04,23.66,32.93,32.77,52.33,0.0,0.44,20.36,20.36,0.0,1.19,11.4,64.84,35.73,69.44,27.92,37.2,0.0,3.61,17.52,23.47,0.0,0.94,25.74,26.88,3.8,10.89,2.09,9.5,5.96,18.74,14.95,23.29,29.65,37.6,3.73,15.41,8.13,34.22,0.0,4.08,34.22,69.44,69.44,69.44,0.0,0.0,6.46,21.61,21.61,40.07,15.8,20.11,45.82,45.82,21.73,23.54,37.76,69.44,62.1,69.44,44.24,51.31,12.41,14.81,35.42,45.3,12.76,14.18,51.93,53.07,27.22,34.31,24.68,26.62,28.21,36.18,32.27,41.2,50.63,57.13,20.41,27.99,34.49,60.58,10.63,12.63,60.58,69.44,69.44,69.44,8.04,8.62,31.62,40.48,41.15,64.08
1,73903,5350353.02,535035302.0,(POLYGON ((324253.4107999997 4844973.746400001...,9.86,12.53,36.42,36.42,14.3,15.88,25.11,69.31,50.02,69.44,37.99,44.58,6.3,9.01,27.83,36.13,7.19,9.8,40.1,41.23,16.82,26.06,13.99,19.42,21.06,27.42,23.23,34.64,42.92,47.85,12.65,21.67,22.41,48.5,6.42,9.27,48.51,69.44,69.44,69.44,3.97,4.89,23.29,32.22,32.12,52.07,10.1,12.85,36.73,36.73,14.65,16.27,25.12,69.44,50.34,69.44,38.51,45.05,6.28,9.01,28.39,36.84,7.3,9.67,41.22,42.36,17.81,26.64,12.48,19.72,21.04,27.97,22.26,34.28,44.14,48.42,12.61,21.62,22.7,48.83,6.49,9.41,48.83,69.44,69.44,69.44,3.82,5.04,23.66,32.93,32.77,52.33,0.0,0.44,20.36,20.36,0.0,1.19,11.4,64.84,35.73

In [24]:
summary_buildings.shape

(394475, 188)

In [32]:
# takes at least an hour to run?
start = time.time()
N = 572
c = 1
for group,df in buildings.groupby('ctnum'):
    #printing out summary for each census tract
    print("Doing CTNUM: ",group, "Number left: ", N-c)
    c+=1
    for current_column in df.columns:
        if current_column.startswith("m"):
            for measure in ['avg','med','min','max']:
                metric,poi,n = current_column.split("_")
                out_column = metric+"_"+poi+"_"+str(n)+"_"+measure
                res = {'avg':round(df[current_column].mean(),2),'med' : round(df[current_column].median(),2),
                       'min': round(df[current_column].min(),2),'max':round(df[current_column].max(),2)}
                #works for cts
                cts.loc[group,out_column] =res[measure] 
                summary_buildings.loc[summary_buildings[summary_buildings.ctnum == group].index,out_column] =res[measure]
                #works for pednet
                pednet_sum.loc[pednet_sum[pednet_sum.ctnum == group].index,out_column] =res[measure]
            #print("Finished Column(s) for: ", current_column)


Doing CTNUM:  535000100.0 Number left:  571


/opt/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


Doing CTNUM:  535000200.0 Number left:  570
Doing CTNUM:  535000300.0 Number left:  569
Doing CTNUM:  535000400.0 Number left:  568
Doing CTNUM:  535000500.0 Number left:  567
Doing CTNUM:  535000701.0 Number left:  566
Doing CTNUM:  535000702.0 Number left:  565
Doing CTNUM:  535000801.0 Number left:  564
Doing CTNUM:  535000802.0 Number left:  563
Doing CTNUM:  535000900.0 Number left:  562
Doing CTNUM:  535001001.0 Number left:  561
Doing CTNUM:  535001002.0 Number left:  560
Doing CTNUM:  535001100.0 Number left:  559
Doing CTNUM:  535001201.0 Number left:  558
Doing CTNUM:  535001203.0 Number left:  557
Doing CTNUM:  535001204.0 Number left:  556
Doing CTNUM:  535001301.0 Number left:  555
Doing CTNUM:  535001302.0 Number left:  554
Doing CTNUM:  535001400.0 Number left:  553
Doing CTNUM:  535001500.0 Number left:  552
Doing CTNUM:  535001600.0 Number left:  551
Doing CTNUM:  535001700.0 Number left:  550
Doing CTNUM:  535001800.0 Number left:  549
Doing CTNUM:  535001900.0 Number

Doing CTNUM:  535016300.0 Number left:  383
Doing CTNUM:  535016400.0 Number left:  382
Doing CTNUM:  535016500.0 Number left:  381
Doing CTNUM:  535016600.0 Number left:  380
Doing CTNUM:  535016701.0 Number left:  379
Doing CTNUM:  535016702.0 Number left:  378
Doing CTNUM:  535016800.0 Number left:  377
Doing CTNUM:  535016901.0 Number left:  376
Doing CTNUM:  535016902.0 Number left:  375
Doing CTNUM:  535017000.0 Number left:  374
Doing CTNUM:  535017100.0 Number left:  373
Doing CTNUM:  535017200.0 Number left:  372
Doing CTNUM:  535017300.0 Number left:  371
Doing CTNUM:  535017400.0 Number left:  370
Doing CTNUM:  535017501.0 Number left:  369
Doing CTNUM:  535017502.0 Number left:  368
Doing CTNUM:  535017600.0 Number left:  367
Doing CTNUM:  535018000.0 Number left:  366
Doing CTNUM:  535018101.0 Number left:  365
Doing CTNUM:  535018102.0 Number left:  364
Doing CTNUM:  535018200.0 Number left:  363
Doing CTNUM:  535018301.0 Number left:  362
Doing CTNUM:  535018302.0 Number

Doing CTNUM:  535030000.0 Number left:  196
Doing CTNUM:  535030101.0 Number left:  195
Doing CTNUM:  535030103.0 Number left:  194
Doing CTNUM:  535030104.0 Number left:  193
Doing CTNUM:  535030201.0 Number left:  192
Doing CTNUM:  535030202.0 Number left:  191
Doing CTNUM:  535030203.0 Number left:  190
Doing CTNUM:  535030300.0 Number left:  189
Doing CTNUM:  535030401.0 Number left:  188
Doing CTNUM:  535030402.0 Number left:  187
Doing CTNUM:  535030403.0 Number left:  186
Doing CTNUM:  535030404.0 Number left:  185
Doing CTNUM:  535030405.0 Number left:  184
Doing CTNUM:  535030406.0 Number left:  183
Doing CTNUM:  535030501.0 Number left:  182
Doing CTNUM:  535030503.0 Number left:  181
Doing CTNUM:  535030504.0 Number left:  180
Doing CTNUM:  535030601.0 Number left:  179
Doing CTNUM:  535030602.0 Number left:  178
Doing CTNUM:  535030703.0 Number left:  177
Doing CTNUM:  535030704.0 Number left:  176
Doing CTNUM:  535030705.0 Number left:  175
Doing CTNUM:  535030706.0 Number

Doing CTNUM:  535037824.0 Number left:  7
Doing CTNUM:  535037825.0 Number left:  6
Doing CTNUM:  535037826.0 Number left:  5
Doing CTNUM:  535037827.0 Number left:  4
Doing CTNUM:  535037828.0 Number left:  3
Doing CTNUM:  535080201.0 Number left:  2
Doing CTNUM:  535080202.0 Number left:  1


In [34]:
summary_buildings[summary_buildings.ctnum == group].index

Int64Index([77216, 78238, 78368, 78378, 78385, 78389, 78393, 78397, 78412,
            78420,
            ...
            89819, 89820, 89821, 89822, 89823, 89824, 89825, 89826, 89827,
            89828],
           dtype='int64', length=1644)

In [35]:
#summary_buildings doesn't have the ctnum column
summary_buildings.dtypes

id                   int64
ctuid              float64
ctnum              float64
geom                object
m_fc_0_avg         float64
m_fc_1_avg         float64
m_ff_0_avg         float64
m_ff_1_avg         float64
m_sm_0_avg         float64
m_sm_1_avg         float64
m_hp_0_avg         float64
m_hp_1_avg         float64
m_sxhc_0_avg       float64
m_sxhc_1_avg       float64
m_wc_0_avg         float64
m_wc_1_avg         float64
m_sc_0_avg         float64
m_sc_1_avg         float64
m_ar_0_avg         float64
m_ar_1_avg         float64
m_dc_0_avg         float64
m_dc_1_avg         float64
m_di_0_avg         float64
m_di_1_avg         float64
m_ems_0_avg        float64
m_ems_1_avg        float64
m_frc_0_avg        float64
m_frc_1_avg        float64
m_fr_0_avg         float64
m_fr_1_avg         float64
m_lb_0_avg         float64
m_lb_1_avg         float64
m_plc_0_avg        float64
m_plc_1_avg        float64
m_rec_0_avg        float64
m_rec_1_avg        float64
m_thing_0_avg      float64
m

In [36]:
summary_buildings.head(2)

,id,ctuid,ctnum,geom,m_fc_0_avg,m_fc_1_avg,m_ff_0_avg,m_ff_1_avg,m_sm_0_avg,m_sm_1_avg,m_hp_0_avg,m_hp_1_avg,m_sxhc_0_avg,m_sxhc_1_avg,m_wc_0_avg,m_wc_1_avg,m_sc_0_avg,m_sc_1_avg,m_ar_0_avg,m_ar_1_avg,m_dc_0_avg,m_dc_1_avg,m_di_0_avg,m_di_1_avg,m_ems_0_avg,m_ems_1_avg,m_frc_0_avg,m_frc_1_avg,m_fr_0_avg,m_fr_1_avg,m_lb_0_avg,m_lb_1_avg,m_plc_0_avg,m_plc_1_avg,m_rec_0_avg,m_rec_1_avg,m_thing_0_avg,m_thing_1_avg,m_vtl_0_avg,m_vtl_1_avg,m_bp_0_avg,m_bp_1_avg,m_bs_0_avg,m_bs_1_avg,m_ttcstop_0_avg,m_ttcstop_1_avg,m_ttcst_0_avg,m_ttcst_1_avg,m_ttcacc_0_avg,m_ttcacc_1_avg,m_fc_0_med,m_fc_1_med,m_ff_0_med,m_ff_1_med,m_sm_0_med,m_sm_1_med,m_hp_0_med,m_hp_1_med,m_sxhc_0_med,m_sxhc_1_med,m_wc_0_med,m_wc_1_med,m_sc_0_med,m_sc_1_med,m_ar_0_med,m_ar_1_med,m_dc_0_med,m_dc_1_med,m_di_0_med,m_di_1_med,m_ems_0_med,m_ems_1_med,m_frc_0_med,m_frc_1_med,m_fr_0_med,m_fr_1_med,m_lb_0_med,m_lb_1_med,m_plc_0_med,m_plc_1_med,m_rec_0_med,m_rec_1_med,m_thing_0_med,m_thing_1_med,m_vtl_0_med,m_vtl_1_med,m_bp_0_med,m_bp_1_med,m_bs_0_med,m_bs_1_med,m_ttcstop_0_med,m_ttcstop_1_med,m_ttcst_0_med,m_ttcst_1_med,m_ttcacc_0_med,m_ttcacc_1_med,m_fc_0_min,m_fc_1_min,m_ff_0_min,m_ff_1_min,m_sm_0_min,m_sm_1_min,m_hp_0_min,m_hp_1_min,m_sxhc_0_min,m_sxhc_1_min,m_wc_0_min,m_wc_1_min,m_sc_0_min,m_sc_1_min,m_ar_0_min,m_ar_1_min,m_dc_0_min,m_dc_1_min,m_di_0_min,m_di_1_min,m_ems_0_min,m_ems_1_min,m_frc_0_min,m_frc_1_min,m_fr_0_min,m_fr_1_min,m_lb_0_min,m_lb_1_min,m_plc_0_min,m_plc_1_min,m_rec_0_min,m_rec_1_min,m_thing_0_min,m_thing_1_min,m_vtl_0_min,m_vtl_1_min,m_bp_0_min,m_bp_1_min,m_bs_0_min,m_bs_1_min,m_ttcstop_0_min,m_ttcstop_1_min,m_ttcst_0_min,m_ttcst_1_min,m_ttcacc_0_min,m_ttcacc_1_min,m_fc_0_max,m_fc_1_max,m_ff_0_max,m_ff_1_max,m_sm_0_max,m_sm_1_max,m_hp_0_max,m_hp_1_max,m_sxhc_0_max,m_sxhc_1_max,m_wc_0_max,m_wc_1_max,m_sc_0_max,m_sc_1_max,m_ar_0_max,m_ar_1_max,m_dc_0_max,m_dc_1_max,m_di_0_max,m_di_1_max,m_ems_0_max,m_ems_1_max,m_frc_0_max,m_frc_1_max,m_fr_0_max,m_fr_1_max,m_lb_0_max,m_lb_1_max,m_plc_0_max,m_plc_1_max,m_rec_0_max,m_rec_1_max,m_thing_0_max,m_thing_1_max,m_vtl_0_max,m_vtl_1_max,m_bp_0_max,m_bp_1_max,m_bs_0_max,m_bs_1_max,m_ttcstop_0_max,m_ttcstop_1_max,m_ttcst_0_max,m_ttcst_1_max,m_ttcacc_0_max,m_ttcacc_1_max
0,73902,5350353.02,535035302.0,"(POLYGON ((323669.8799999999 4844969.168, 3236...",9.86,12.53,36.42,36.42,14.3,15.88,25.11,69.31,50.02,69.44,37.99,44.58,6.3,9.01,27.83,36.13,7.19,9.8,40.1,41.23,16.82,26.06,13.99,19.42,21.06,27.42,23.23,34.64,42.92,47.85,12.65,21.67,22.41,48.5,6.42,9.27,48.51,69.44,69.44,69.44,3.97,4.89,23.29,32.22,32.12,52.07,10.1,12.85,36.73,36.73,14.65,16.27,25.12,69.44,50.34,69.44,38.51,45.05,6.28,9.01,28.39,36.84,7.3,9.67,41.22,42.36,17.81,26.64,12.48,19.72,21.04,27.97,22.26,34.28,44.14,48.42,12.61,21.62,22.7,48.83,6.49,9.41,48.83,69.44,69.44,69.44,3.82,5.04,23.66,32.93,32.77,52.33,0.0,0.44,20.36,20.36,0.0,1.19,11.4,64.84,35.73,69.44,27.92,37.2,0.0,3.61,17.52,23.47,0.0,0.94,25.74,26.88,3.8,10.89,2.09,9.5,5.96,18.74,14.95,23.29,29.65,37.6,3.73,15.41,8.13,34.22,0.0,4.08,34.22,69.44,69.44,69.44,0.0,0.0,6.46,21.61,21.61,40.07,15.8,20.11,45.82,45.82,21.73,23.54,37.76,69.44,62.1,69.44,44.24,51.31,12.41,14.81,35.42,45.3,12.76,14.18,51.93,53.07,27.22,34.31,24.68,26.62,28.21,36.18,32.27,41.2,50.63,57.13,20.41,27.99,34.49,60.58,10.63,12.63,60.58,69.44,69.44,69.44,8.04,8.62,31.62,40.48,41.15,64.08
1,73903,5350353.02,535035302.0,(POLYGON ((324253.4107999997 4844973.746400001...,9.86,12.53,36.42,36.42,14.3,15.88,25.11,69.31,50.02,69.44,37.99,44.58,6.3,9.01,27.83,36.13,7.19,9.8,40.1,41.23,16.82,26.06,13.99,19.42,21.06,27.42,23.23,34.64,42.92,47.85,12.65,21.67,22.41,48.5,6.42,9.27,48.51,69.44,69.44,69.44,3.97,4.89,23.29,32.22,32.12,52.07,10.1,12.85,36.73,36.73,14.65,16.27,25.12,69.44,50.34,69.44,38.51,45.05,6.28,9.01,28.39,36.84,7.3,9.67,41.22,42.36,17.81,26.64,12.48,19.72,21.04,27.97,22.26,34.28,44.14,48.42,12.61,21.62,22.7,48.83,6.49,9.41,48.83,69.44,69.44,69.44,3.82,5.04,23.66,32.93,32.77,52.33,0.0,0.44,20.36,20.36,0.0,1.19,11.4,64.84,35.73

In [39]:
cts.head(2)

,gid,__gid,ctuid,ctname,pruid,prname,cmauid,cmapuid,cmaname,cmatype,shape_leng,shape_area,geotext,geom,m_fc_0_avg,m_fc_1_avg,m_ff_0_avg,m_ff_1_avg,m_sm_0_avg,m_sm_1_avg,m_hp_0_avg,m_hp_1_avg,m_sxhc_0_avg,m_sxhc_1_avg,m_wc_0_avg,m_wc_1_avg,m_sc_0_avg,m_sc_1_avg,m_ar_0_avg,m_ar_1_avg,m_dc_0_avg,m_dc_1_avg,m_di_0_avg,m_di_1_avg,m_ems_0_avg,m_ems_1_avg,m_frc_0_avg,m_frc_1_avg,m_fr_0_avg,m_fr_1_avg,m_lb_0_avg,m_lb_1_avg,m_plc_0_avg,m_plc_1_avg,m_rec_0_avg,m_rec_1_avg,m_thing_0_avg,m_thing_1_avg,m_vtl_0_avg,m_vtl_1_avg,m_bp_0_avg,m_bp_1_avg,m_bs_0_avg,m_bs_1_avg,m_ttcstop_0_avg,m_ttcstop_1_avg,m_ttcst_0_avg,m_ttcst_1_avg,m_ttcacc_0_avg,m_ttcacc_1_avg,m_fc_0_med,m_fc_1_med,m_ff_0_med,m_ff_1_med,m_sm_0_med,m_sm_1_med,m_hp_0_med,m_hp_1_med,m_sxhc_0_med,m_sxhc_1_med,m_wc_0_med,m_wc_1_med,m_sc_0_med,m_sc_1_med,m_ar_0_med,m_ar_1_med,m_dc_0_med,m_dc_1_med,m_di_0_med,m_di_1_med,m_ems_0_med,m_ems_1_med,m_frc_0_med,m_frc_1_med,m_fr_0_med,m_fr_1_med,m_lb_0_med,m_lb_1_med,m_plc_0_med,m_plc_1_med,m_rec_0_med,m_rec_1_med,m_thing_0_med,m_thing_1_med,m_vtl_0_med,m_vtl_1_med,m_bp_0_med,m_bp_1_med,m_bs_0_med,m_bs_1_med,m_ttcstop_0_med,m_ttcstop_1_med,m_ttcst_0_med,m_ttcst_1_med,m_ttcacc_0_med,m_ttcacc_1_med,m_fc_0_min,m_fc_1_min,m_ff_0_min,m_ff_1_min,m_sm_0_min,m_sm_1_min,m_hp_0_min,m_hp_1_min,m_sxhc_0_min,m_sxhc_1_min,m_wc_0_min,m_wc_1_min,m_sc_0_min,m_sc_1_min,m_ar_0_min,m_ar_1_min,m_dc_0_min,m_dc_1_min,m_di_0_min,m_di_1_min,m_ems_0_min,m_ems_1_min,m_frc_0_min,m_frc_1_min,m_fr_0_min,m_fr_1_min,m_lb_0_min,m_lb_1_min,m_plc_0_min,m_plc_1_min,m_rec_0_min,m_rec_1_min,m_thing_0_min,m_thing_1_min,m_vtl_0_min,m_vtl_1_min,m_bp_0_min,m_bp_1_min,m_bs_0_min,m_bs_1_min,m_ttcstop_0_min,m_ttcstop_1_min,m_ttcst_0_min,m_ttcst_1_min,m_ttcacc_0_min,m_ttcacc_1_min,m_fc_0_max,m_fc_1_max,m_ff_0_max,m_ff_1_max,m_sm_0_max,m_sm_1_max,m_hp_0_max,m_hp_1_max,m_sxhc_0_max,m_sxhc_1_max,m_wc_0_max,m_wc_1_max,m_sc_0_max,m_sc_1_max,m_ar_0_max,m_ar_1_max,m_dc_0_max,m_dc_1_max,m_di_0_max,m_di_1_max,m_ems_0_max,m_ems_1_max,m_frc_0_max,m_frc_1_max,m_fr_0_max,m_fr_1_max,m_lb_0_max,m_lb_1_max,m_plc_0_max,m_plc_1_max,m_rec_0_max,m_rec_1_max,m_thing_0_max,m_thing_1_max,m_vtl_0_max,m_vtl_1_max,m_bp_0_max,m_bp_1_max,m_bs_0_max,m_bs_1_max,m_ttcstop_0_max,m_ttcstop_1_max,m_ttcst_0_max,m_ttcst_1_max,m_ttcacc_0_max,m_ttcacc_1_max
ctnum,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
535000100.0,1,62,5350001.00,0001.00,35,Ontario,535,35535,Toronto,B,36906.495474,6.619690e+06,535000100,(POLYGON ((320588.9306980086 4835855.647789237...,12.98,15.87,20.10,20.10,13.85,18.58,41.57,53.06,60.39,66.17,21.93,33.43,15.31,19.78,45.81,67.69,15.68,19.07,28.13,31.94,27.71,48.07,16.58,19.55,22.38,30.34,24.06,30.52,28.73,49.08,19.4,21.25,32.74,40.87,14.70,18.02,24.32,28.06,60.64,69.44,6.07,6.78,44.6,46.67,45.46,49.66,10.16,13.65,19.03,19.03,14.06,17.56,40.45,52.13,60.58,69.44,17.70,31.77,15.45,19.25,44.63,69.44,14.55,17.03,26.54,29.58,25.45,47.63,14.46,19.00,19.95,26.87,23.54,28.32,29.67,48.48,17.29,20.67,30.10,37.11,14.10,16.75,23.51,26.43,60.13,69.44,3.93,4.52,42.87,46.16,42.92,48.14,0.00,1.76,0.00,0.00,0.00,1.58,19.19,36.02,43.05,52.55,0.60,18.42,2.75,6.91,28.23,53.79,2.75,5.46,7.32,7.56,8.48,35.16,2.75,5.12,6.84,18.14,7.17,15.61,14.55,34.25,0.00,7.63,9.11,18.87,2.75,6.58,6.75,7.56,39.98,69.44,0.00,0.00,33.87,35.32,34.74,36.43,46.81,48.32,56.67,56.67,27.99,43.20,69.44,69.44,69.44,69.44,49.77,62.55,39.92,41.93,69.44,69.44,42.88,47.60,56.06,58.25,63.96,68.69,47.05,47.79,50.94,55.61,54.11,60.07,52.14,69.44,47.79,47.79,62.55,69.44,38.75,42.78,51.61,52.85,69.44,69.44,27.63,27.89,67.99,69.44,69.44,69.44
535000200.0,2,63,5350002.00,0002.00,35,Ontario,535,35535,Toronto,B,32534.307654,3.293422e+06,535000200,"(POLYGON ((316642.831678931 4832470.554474635,...",68.98,69.02,69.19,69.19,69.06,69.08,69.18,69.19,69.24,69.39,69.08,69.11,33.24,68.98,69.34,69.40,11.75,38.93,69.10,

In [40]:
pednet_sum.head(2)

,gid_left,ctuid,ctnum,geom,m_fc_0_avg,m_fc_1_avg,m_ff_0_avg,m_ff_1_avg,m_sm_0_avg,m_sm_1_avg,m_hp_0_avg,m_hp_1_avg,m_sxhc_0_avg,m_sxhc_1_avg,m_wc_0_avg,m_wc_1_avg,m_sc_0_avg,m_sc_1_avg,m_ar_0_avg,m_ar_1_avg,m_dc_0_avg,m_dc_1_avg,m_di_0_avg,m_di_1_avg,m_ems_0_avg,m_ems_1_avg,m_frc_0_avg,m_frc_1_avg,m_fr_0_avg,m_fr_1_avg,m_lb_0_avg,m_lb_1_avg,m_plc_0_avg,m_plc_1_avg,m_rec_0_avg,m_rec_1_avg,m_thing_0_avg,m_thing_1_avg,m_vtl_0_avg,m_vtl_1_avg,m_bp_0_avg,m_bp_1_avg,m_bs_0_avg,m_bs_1_avg,m_ttcstop_0_avg,m_ttcstop_1_avg,m_ttcst_0_avg,m_ttcst_1_avg,m_ttcacc_0_avg,m_ttcacc_1_avg,m_fc_0_med,m_fc_1_med,m_ff_0_med,m_ff_1_med,m_sm_0_med,m_sm_1_med,m_hp_0_med,m_hp_1_med,m_sxhc_0_med,m_sxhc_1_med,m_wc_0_med,m_wc_1_med,m_sc_0_med,m_sc_1_med,m_ar_0_med,m_ar_1_med,m_dc_0_med,m_dc_1_med,m_di_0_med,m_di_1_med,m_ems_0_med,m_ems_1_med,m_frc_0_med,m_frc_1_med,m_fr_0_med,m_fr_1_med,m_lb_0_med,m_lb_1_med,m_plc_0_med,m_plc_1_med,m_rec_0_med,m_rec_1_med,m_thing_0_med,m_thing_1_med,m_vtl_0_med,m_vtl_1_med,m_bp_0_med,m_bp_1_med,m_bs_0_med,m_bs_1_med,m_ttcstop_0_med,m_ttcstop_1_med,m_ttcst_0_med,m_ttcst_1_med,m_ttcacc_0_med,m_ttcacc_1_med,m_fc_0_min,m_fc_1_min,m_ff_0_min,m_ff_1_min,m_sm_0_min,m_sm_1_min,m_hp_0_min,m_hp_1_min,m_sxhc_0_min,m_sxhc_1_min,m_wc_0_min,m_wc_1_min,m_sc_0_min,m_sc_1_min,m_ar_0_min,m_ar_1_min,m_dc_0_min,m_dc_1_min,m_di_0_min,m_di_1_min,m_ems_0_min,m_ems_1_min,m_frc_0_min,m_frc_1_min,m_fr_0_min,m_fr_1_min,m_lb_0_min,m_lb_1_min,m_plc_0_min,m_plc_1_min,m_rec_0_min,m_rec_1_min,m_thing_0_min,m_thing_1_min,m_vtl_0_min,m_vtl_1_min,m_bp_0_min,m_bp_1_min,m_bs_0_min,m_bs_1_min,m_ttcstop_0_min,m_ttcstop_1_min,m_ttcst_0_min,m_ttcst_1_min,m_ttcacc_0_min,m_ttcacc_1_min,m_fc_0_max,m_fc_1_max,m_ff_0_max,m_ff_1_max,m_sm_0_max,m_sm_1_max,m_hp_0_max,m_hp_1_max,m_sxhc_0_max,m_sxhc_1_max,m_wc_0_max,m_wc_1_max,m_sc_0_max,m_sc_1_max,m_ar_0_max,m_ar_1_max,m_dc_0_max,m_dc_1_max,m_di_0_max,m_di_1_max,m_ems_0_max,m_ems_1_max,m_frc_0_max,m_frc_1_max,m_fr_0_max,m_fr_1_max,m_lb_0_max,m_lb_1_max,m_plc_0_max,m_plc_1_max,m_rec_0_max,m_rec_1_max,m_thing_0_max,m_thing_1_max,m_vtl_0_max,m_vtl_1_max,m_bp_0_max,m_bp_1_max,m_bs_0_max,m_bs_1_max,m_ttcstop_0_max,m_ttcstop_1_max,m_ttcst_0_max,m_ttcst_1_max,m_ttcacc_0_max,m_ttcacc_1_max
0,107403,5350001.00,535000100.0,"(LINESTRING (318125.824 4833673.8743, 318551.8...",12.98,15.87,20.1,20.1,13.85,18.58,41.57,53.06,60.39,66.17,21.93,33.43,15.31,19.78,45.81,67.69,15.68,19.07,28.13,31.94,27.71,48.07,16.58,19.55,22.38,30.34,24.06,30.52,28.73,49.08,19.4,21.25,32.74,40.87,14.7,18.02,24.32,28.06,60.64,69.44,6.07,6.78,44.6,46.67,45.46,49.66,10.16,13.65,19.03,19.03,14.06,17.56,40.45,52.13,60.58,69.44,17.7,31.77,15.45,19.25,44.63,69.44,14.55,17.03,26.54,29.58,25.45,47.63,14.46,19.0,19.95,26.87,23.54,28.32,29.67,48.48,17.29,20.67,30.1,37.11,14.1,16.75,23.51,26.43,60.13,69.44,3.93,4.52,42.87,46.16,42.92,48.14,0.0,1.76,0.0,0.0,0.0,1.58,19.19,36.02,43.05,52.55,0.6,18.42,2.75,6.91,28.23,53.79,2.75,5.46,7.32,7.56,8.48,35.16,2.75,5.12,6.84,18.14,7.17,15.61,14.55,34.25,0.0,7.63,9.11,18.87,2.75,6.58,6.75,7.56,39.98,69.44,0.0,0.0,33.87,35.32,34.74,36.43,46.81,48.32,56.67,56.67,27.99,43.2,69.44,69.44,69.44,69.44,49.77,62.55,39.92,41.93,69.44,69.44,42.88,47.6,56.06,58.25,63.96,68.69,47.05,47.79,50.94,55.61,54.11,60.07,52.14,69.44,47.79,47.79,62.55,69.44,38.75,42.78,51.61,52.85,69.44,69.44,27.63,27.89,67.99,69.44,69.44,69.44
80,117847,5350001.00,535000100.0,"(LINESTRING (317559.4142 4834849.1415, 317606....",12.98,15.87,20.1,20.1,13.85,18.58,41.57,53.06,60.39,66.17,21.93,33.43,15.31,19.78,45.81,67.69,15.68,19.07,28.13,31.94,27.71,48.07,16.58,19.55,22.38,30.34,24.06,30.52,28.73,49.08,19.4,21.25,32.74,40.87,14.7,18.02,24.32,28.06,60.64,69.44,6.07,6.78,44.6,46.67,45.46,49.66,10.16,13.65,19.03,19.03,14.06,17.56,40.45,52.13,60.58,69.44,17.7,31.77,15.45,19.25,44.63,69.44,14.55,17.03,26.54,29.58,25.45,47.63,14.46,19.0,19.95,26.87,23.54,28.32,29.67,48.48,17.29,20.67,30.1,37.11,14.1,16.75,23.51,26.43,60.13,69.44,3.93,4.52,42.87,46.16,42.92,48.14,0.0,1.76,0.

In [41]:
#You always need to drop the geometry column and write a new WKT text geometry column 

cts1 = cts.copy()
cts1['geometry'] = cts1['geom']
cts1['geom'] = cts1['geometry'].apply(lambda x: WKTElement(x.wkt, srid=2019))
#drop the geometry column as it is now duplicative
cts1.drop('geometry', 1, inplace=True)
#Adds objectid as double. Need to change to integer
cts1['objectid'] = cts1.index.astype(np.int64)
cts1.to_sql('census_summary_walktimes', walkability_engine, schema='results', if_exists='replace', index=True, 
                         dtype={'geom': Geometry('MULTIPOLYGON', srid=2019)})

In [42]:
#keeping only valid polygon geometries
summary_buildings_out = summary_buildings[summary_buildings.is_valid]

In [48]:
#You always need to drop the geometry column and write a new WKT text geometry column
summary_buildings1 = summary_buildings_out.copy()
summary_buildings1['geometry'] = summary_buildings1['geom']
summary_buildings1['geom'] = summary_buildings1['geometry'].apply(lambda x: WKTElement(x.wkt, srid=2019))
#drop the geometry column as it is now duplicative
summary_buildings1.drop('geometry', 1, inplace=True)
#Added objectid using the following sql so ArcGIS Pro can read objectid
#set objectid as a primary key afterwards

#CREATE SEQUENCE results.objectid_seq OWNED BY results.buildings_summary_walktimes_fin_v.objectid;
#update results.buildings_summary_walktimes_fin_v set objectid = nextval('results.objectid_seq');

#Adds objectid as double.
#summary_buildings1['objectid'] = summary_buildings1.index.astype(np.int64)
#summary_buildings1['objectid'] = list(range(1,summary_buildings1.shape[0]))
summary_buildings1.to_sql('buildings_summary_walktimes',walkability_engine, schema='results', if_exists='replace', index=True, 
                         dtype={'geom': Geometry('MULTIPOLYGON', srid=2019)})

In [44]:

#You always need to drop the geometry column and write a new WKT text geometry column 
pednet_sum1 = pednet_sum.copy()
pednet_sum1['geometry'] = pednet_sum1['geom']
pednet_sum1['geom'] = pednet_sum1['geometry'].apply(lambda x: WKTElement(x.wkt, srid=2019))
#drop the geometry column as it is now duplicative
pednet_sum1.drop('geometry', 1, inplace=True)
#Adds objectid as double.
#objectid = 9306 is duplicate
#pednet_sum1['objectid'] = list(range(1,pednet_sum1.shape[0]))
pednet_sum1.to_sql('pednet_summary_walktimes', walkability_engine, schema='results', if_exists='replace', index=True, 
                         dtype={'geom': Geometry('MULTILINESTRING', srid=2019)})